In [16]:
import numpy as np
from astropy.table import Table
import subprocess
import os

In [2]:
#Read the full database
dbase = Table.read("dbase.fits")

In [3]:
#Set objects we want to focus on
wids = [
    "W0019-1046",
    "W0116-0505",
    "W0204-0506",
    "W0220+0137",
    "W0831+0140",
]
wids = np.array(wids, dtype=bytes)

In [4]:
#Limit the database to only the objects of interest. 
dbase = dbase[np.isin(dbase['Short Name'],wids)]

In [5]:
phot = Table()
phot['WID'] = dbase['Short Name']

In [6]:
phot['zspec'] = np.where(dbase['zspec refined'].mask, dbase['zspec'], dbase['zspec refined'])

In [7]:
#Set the bands we want in the form we need them. 
cols = [
    'w1mpro (CatWISE)',
    'w1sigmpro (CatWISE)',
    'w2mpro (CatWISE)',
    'w2sigmpro (CatWISE)',
    'w3mpro',
    'w3sigmpro',
    'w4mpro',
    'w4sigmpro',
    # 'gMeanPSFMag (PS1)',
    # 'gMeanPSFMagErr (PS1)',
    # 'rMeanPSFMag (PS1)',
    # 'rMeanPSFMagErr (PS1)',
    # 'iMeanPSFMag (PS1)',
    # 'iMeanPSFMagErr (PS1)',
    # 'zMeanPSFMag (PS1)',
    # 'zMeanPSFMagErr (PS1)',
    # 'yMeanPSFMag (PS1)',
    # 'yMeanPSFMagErr',
    'gm (Legacy DR10)',
    'gm_e (Legacy DR10)',
    'rm (Legacy DR10)',
    'rm_e (Legacy DR10)',
    'im (Legacy DR10)',
    'im_e (Legacy DR10)',
    'zm (Legacy DR10)',
    'zm_e (Legacy DR10)',
    # 'ypetromag (VHS DR5)',
    # 'ypetromagerr (VHS DR5)',
    # 'jpetromag (VHS DR5)',
    # 'jpetromagerr (VHS DR5)',
    # 'hpetromag (VHS DR5)',
    # 'hpetromagerr (VHS DR5)',
    # 'kspetromag (VHS DR5)',
    # 'kspetromagerr (VHS DR5)',
    # 'ypetromag (UKIDSS DR11)',
    # 'ypetromagerr (UKIDSS DR11)',
    # 'jpetromag (UKIDSS DR11)',
    # 'jpetromagerr (UKIDSS DR11)',
    # 'hpetromag (UKIDSS DR11)',
    # 'hpetromagerr (UKIDSS DR11)',
    # 'kpetromag (UKIDSS DR11)',
    # 'kpetromagerr (UKIDSS DR11)',
    'targeted r mag',
    'targeted r magerr',
    'targeted g mag',
    'targeted g magerr',
    'targeted J mag',
    'targeted J magerr',
    'targeted H mag',
    'targeted H magerr',
    'targeted KS mag',
    'targeted KS magerr',
    'i1mag',
    'i1magerr',
    'i2mag',
    'i2magerr',
]
for col in cols:
    phot[col] = dbase[col]
    phot[col].fill_value = -99.

In [8]:
#Save the photometry file. 
phot.filled().write("phot.dat", format='ascii.no_header', overwrite=True)

In [11]:
#Save the number of bands we are using. 
cato = open("nchan.txt","w")
cato.write("{}\n".format(len(cols[::2])))
cato.close()

In [12]:
filters = {
    'w1mpro (CatWISE)'       : {"bname":"real_wise_band_1","norm":1,"zp":306.681},
    'w2mpro (CatWISE)'       : {"bname":"real_wise_band_2","norm":1,"zp":170.663},
    'w3mpro'                 : {"bname":"real_wise_band_3","norm":1,"zp":29.0448},
    'w4mpro'                 : {"bname":"real_wise_band_4","norm":1,"zp":8.2839},
    'gMeanPSFMag (PS1)'      : {"bname":"gp1","norm":3,"zp":3631},
    'rMeanPSFMag (PS1)'      : {"bname":"rp1","norm":3,"zp":3631},
    'iMeanPSFMag (PS1)'      : {"bname":"ip1","norm":3,"zp":3631},
    'zMeanPSFMag (PS1)'      : {"bname":"zp1","norm":3,"zp":3631},
    'yMeanPSFMag (PS1)'      : {"bname":"yp1","norm":3,"zp":3631},
    'gm (Legacy DR10)'       : {"bname":"DECAM_g","norm":3,"zp":3631},
    'rm (Legacy DR10)'       : {"bname":"DECAM_r","norm":3,"zp":3631},
    'im (Legacy DR10)'       : {"bname":"DECAM_i","norm":3,"zp":3631},
    'zm (Legacy DR10)'       : {"bname":"DECAM_z","norm":3,"zp":3631},
    'ypetromag (VHS DR5)'    : {"bname":"vistaY" ,"norm":3,"zp":3631},
    'jpetromag (VHS DR5)'    : {"bname":"vistaJ" ,"norm":3,"zp":3631},
    'hpetromag (VHS DR5)'    : {"bname":"vistaH" ,"norm":3,"zp":3631},
    'kspetromag (VHS DR5)'   : {"bname":"vistaKs" ,"norm":3,"zp":3631},
    'ypetromag (UKIDSS DR11)': {"bname":"ukidssy" ,"norm":3,"zp":3631},
    'jpetromag (UKIDSS DR11)': {"bname":"ukidssj" ,"norm":3,"zp":3631},
    'hpetromag (UKIDSS DR11)': {"bname":"ukidssh" ,"norm":3,"zp":3631},
    'kpetromag (UKIDSS DR11)': {"bname":"ukidssk" ,"norm":3,"zp":3631},
    'targeted r mag'         : {"bname":"sdssr","norm":3,"zp":3631},
    'targeted g mag'         : {"bname":"sdssg","norm":3,"zp":3631},
    'targeted J mag'         : {"bname":"jband","norm":1,"zp":1631.00},
    'targeted H mag'         : {"bname":"hband","norm":1,"zp":1024.00},
    'targeted KS mag'        : {"bname":"kband","norm":1,"zp":675.70},
    'i1mag'                  : {"bname":"IRAC_ch1_total_response","norm":2,"zp":277.50},
    'i2mag'                  : {"bname":"IRAC_ch2_total_response","norm":2,"zp":179.50},
}

In [13]:
#Write the bandmag.dat file. 
cato = open("bandmag.dat","w")
cato.write("##\n")
cato.write("#Filter Name                   Norm       Zero Point\n")
cato.write("##\n")
for col in cols[::2]:
    filt = filters[col]
    cato.write("{:30s} {} {:18.3f}\n".format(filt['bname'], filt['norm'], filt['zp']))
cato.close()

In [14]:
subprocess.call("make single", shell=True)
subprocess.call("./single")

gfortran -ffixed-line-length-none -std=legacy  -o single single.f kca_agn_logred.f my_nnls_2.f


my_nnls_2.f:709:43:

  709 |       CALL H12(1,NPP1,NPP1+1,M,A(1,J),1,UP,DUMMY,1,1,0)
      |                                           1
-macosx_version_min has been renamed to -macos_version_min
ld: warning: ignoring duplicate libraries: '-lemutls_w', '-lgcc'


 Reading Filters and Setting Normalizations
 setting up           15  bands 
 corfac            1   5.6649139083033955        3.2916844579120621     
 corfac            2   5.8982764137547319        4.5427003364803635     
 corfac            3   4.6550266769077489        10.565650389253411     
 corfac            4   18.793446753997685        21.915713028010327     
 corfac            5   6.2468062108215445       0.47529740823795252     
 corfac            6   5.4651757988877243       0.63743696121429927     
 corfac            7   5.8243417152457546       0.77782794190173410     
 corfac            8   7.4780167010060339       0.91387936007753645     
 corfac            9   7.8830776602735648       0.61404817823967395     
 corfac           10   7.0018738511324639       0.46259038179778089     
 corfac           11   7.6531843169079092        1.2285286846915080     
 corfac           12   6.5848238789418021        1.6389542143490212     
 corfac           13   8.2802521621269012      

 W0019-1046                     9
 W0116-0505                    11
 W0204-0506                    12
 W0220+0137                    12
 W0831+0140                    11
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG


0

In [15]:
subprocess.call("make double", shell=True)
subprocess.call("./double")

gfortran -ffixed-line-length-none -std=legacy  -o double double.f kca_agn_logred.f my_nnls_2.f


my_nnls_2.f:709:43:

  709 |       CALL H12(1,NPP1,NPP1+1,M,A(1,J),1,UP,DUMMY,1,1,0)
      |                                           1
-macosx_version_min has been renamed to -macos_version_min
ld: warning: ignoring duplicate libraries: '-lemutls_w', '-lgcc'


 Reading Filters and Setting Normalizations
 setting up           15  bands 
 corfac            1   5.6649139083033955        3.2916844579120621     
 corfac            2   5.8982764137547319        4.5427003364803635     
 corfac            3   4.6550266769077489        10.565650389253411     
 corfac            4   18.793446753997685        21.915713028010327     
 corfac            5   6.2468062108215445       0.47529740823795252     
 corfac            6   5.4651757988877243       0.63743696121429927     
 corfac            7   5.8243417152457546       0.77782794190173410     
 corfac            8   7.4780167010060339       0.91387936007753645     
 corfac            9   7.8830776602735648       0.61404817823967395     
 corfac           10   7.0018738511324639       0.46259038179778089     
 corfac           11   7.6531843169079092        1.2285286846915080     
 corfac           12   6.5848238789418021        1.6389542143490212     
 corfac           13   8.2802521621269012      

 W0019-1046                     9
 W0116-0505                    11
 W0204-0506                    12
 W0220+0137                    12
 W0831+0140                    11
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_UNDERFLOW_FLAG


0

In [24]:
os.chdir("f_stat")
subprocess.call("python compare.py", shell=True)
os.chdir("..")